Data exploration - Akira 

In [17]:
!pip3 install pyspellchecker 

  Obtaining dependency information for pyspellchecker from https://files.pythonhosted.org/packages/99/8e/7c79443d302a80cfd59bc365938d51e36e7e9aa7ce8ab1d8a0ca0c8e6065/pyspellchecker-0.8.2-py3-none-any.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 44.7 MB/s eta 0:00:0000:0100:01


In [111]:
import re
import nltk
from spellchecker import SpellChecker
import pandas as pd
from collections import Counter
from nltk.tokenize import sent_tokenize, word_tokenize
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation


In [112]:
df = pd.read_csv('/Users/akirakuno/Downloads/CSSci/year2/speeches2014-2024/speeches_2023-2024.csv') 

In [113]:
df.head() 

,file_id,speaker_name,speaker_party,speech_text,jaar,date,kamer,category,title,document_number,url,meta_url,vergadernummer
0,h-tk-20232024-89-6,De heer Bosma,PVV,De\nAan de orde is de Wijziging Omgevingswet e...,2023-2024,2024-06-26,tk,handelingen,6 Wet uitvoering verordening ontbossingsvrije ...,NaN,https://zoek.officielebekendmakingen.nl/h-tk-2...,https://zoek.officielebekendmakingen.nl/h-tk-2...,"nr. 89, item 6"
1,h-tk-20232024-89-6,De heer Bosma,PVV,De\nAls eerste geef ik graag het woord aan mev...,2023-2024,2024-06-26,tk,handelingen,6 Wet uitvoering verordening ontbossingsvrije ...,NaN,https://zoek.officielebekendmakingen.nl/h-tk-2...,https://zoek.officielebekendmakingen.nl/h-tk-2...,"nr. 89, item 6"
2,h-tk-20232024-89-6,Mevrouw Hertzberger,NSC,Mevrouw\nVoorzitter. Deze wet voor de implemen...,2023-2024,2024-06-26,tk,handelingen,6 Wet uitvoering verordening ontbossingsvrije ...,NaN,https://zoek.officielebekendmakingen.nl/h-tk-2...,https://zoek.officielebekendmakingen.nl/h-tk-2...,"nr. 89, item 6"
3,h-tk-20232024-89-6,De heer Bosma,PVV,De\nNeem even een pauze. Mevrouw Bromet praat ...,2023-2024,2024-06-26,tk,handelingen,6 Wet uitvoering verordening ontbossingsvrije ...,NaN,https://zoek.officielebekendmakingen.nl/h-tk-2...,https://zoek.officielebekendmakingen.nl/h-tk-2...,"nr. 89, item 6"
4,h-tk-20232024-89-6,Mevrouw Bromet,GroenLinks-PvdA,"Mevrouw\nIk verstond het niet, maar dat geeft ...",2023-2024,2024-06-26,tk,handelingen,6 Wet uitvoering verordening ontbossingsvrije ...,NaN,https://zoek.officielebekendmakingen.nl/h-tk-2...,https://zoek.officielebekendmakingen.nl/h-tk-2...,"nr. 89, item 6"


In [114]:
df['speech_text'].head(6) 

0    De\nAan de orde is de Wijziging Omgevingswet e...
1    De\nAls eerste geef ik graag het woord aan mev...
2    Mevrouw\nVoorzitter. Deze wet voor de implemen...
3    De\nNeem even een pauze. Mevrouw Bromet praat ...
4    Mevrouw\nIk verstond het niet, maar dat geeft ...
5                   De\nNee, u wil het ook niet horen.
Name: speech_text, dtype: object

In [115]:
df['speech_text'][0]


'De\nAan de orde is de Wijziging Omgevingswet en enige andere wetten, de Wet uitvoering verordening ontbossingsvrije grondstoffen en producten, Kamerstuk 36518. Een hartelijk woord van welkom aan de minister voor Natuur en Stikstof. Er zijn zeven sprekers van de zijde van de Kamer.\nDe algemene beraadslaging wordt geopend.'

In [116]:
df['speech_text'].isnull().sum() 

0

In [117]:
df['speech_text'] = df['speech_text'].str.replace(r'\n', ' ', regex=True)

Every single starts with either "De", "Mevrouw", or "Minister" which represents the speaker of the text. However, it does not appear to the actual document so it will be taken out. 

In [118]:
def clean_starting_words(text): 
    return re.sub(r'^(De |Mevrouw |Minister )', ' ', text) 

df['speech_text'] = df['speech_text'].apply(clean_starting_words) 

In [119]:
from spellchecker import SpellChecker 

Each dataset maybe too large to run, even after two days, cleaning function still hasn't run

In [120]:
count = (df['title'] =='6 Wet uitvoering verordening ontbossingsvrije grondstoffen en producten').sum()
print(count)


106


so there are 106 back and forth conversation in the first document. 

In [11]:
import PyPDF2
import re 

In [12]:
pip install pdfplumber 

Note: you may need to restart the kernel to use updated packages.


In [13]:
import pdfplumber 
pdf_path = '/Users/akirakuno/Downloads/h-tk-20232024-89-6.pdf' 

def extract_text_from_pdf(pdf_path): 
    text = [] 
    with pdfplumber.open(pdf_path) as pdf: 
        for page in pdf.pages: 
            text.append(page.extract_text()) 
    return text 

pdf_text = extract_text_from_pdf(pdf_path) 

# Combine into a single string for comparison
pdf_full_text = " ".join(pdf_text)

# Display first few characters for verification
print(pdf_full_text[:1000])  # Print first 1000 characters

leide producten zoals leer, chocola, papier en pulp. Door
6 de herkomst van deze producten inzichtelijk te maken tot
op hectareniveau beoogt de verordening ontbossing tegen
Wet uitvoering verordening ontbos-
te gaan. Ik wil benadrukken dat mijn fractie inziet dat
singsvrije grondstoffen en producten bedrijven dankzij deze wetten serieuzer aan de slag moeten
met hun toeleveringsketen. Dat is goed. Zo zorgen we
Wet uitvoering verordening ontbossingsvrije grondstoffen ervoor dat hun producten niet elders op de wereld leiden
en producten tot natuurschade, milieuvervuiling, klimaatimpact, mensen-
rechtenschendingen of anderszins schadelijk zijn. Wel
Aan de orde is de behandeling van: maken wij ons zorgen dat de regeldruk hier disproportioneel
- het wetsvoorstel Wijziging Omgevingswet en enige groot kan worden, met name voor het midden- en kleinbe-
andere wetten (Wet uitvoering verordening ontbossings- drijf. In Europa werd het ook wel een "bureaucratisch
vrije grondstoffen en producten) (36

In [14]:
# Assuming your dataset is already loaded as df
dataset_full_text = " ".join(df["speech_text"][:106].astype(str))

# Display first few characters for verification
print(dataset_full_text[:1000])  # Print first 1000 characters


 Aan de orde is de Wijziging Omgevingswet en enige andere wetten, de Wet uitvoering verordening ontbossingsvrije grondstoffen en producten, Kamerstuk 36518. Een hartelijk woord van welkom aan de minister voor Natuur en Stikstof. Er zijn zeven sprekers van de zijde van de Kamer. De algemene beraadslaging wordt geopend.  Als eerste geef ik graag het woord aan mevrouw Hertzberger van de fractie van Nieuw Sociaal Contract.  Voorzitter. Deze wet voor de implementatie van de Verordening ontbossingsvrije producten en grondstoffen stond vorige week als hamerstuk geagendeerd, maar wij dachten: het afscheid van de minister kunnen we toch niet over ons hart laten gaan; we willen nog een toegift. Maar even serieus: wij hebben dit onthamerd, ondanks het feit dat dit een kwestie is van een zuivere implementatie, een een-op-een-overname van een Europese verordening in Nederlandse wetgeving. Deze Europese wetgeving gaat mogelijk positieve gevolgen hebben voor de biodiversiteit wereldwijd. Mijn fractie

In [15]:
if dataset_full_text.strip() == pdf_full_text.strip():
    print("✅ The dataset follows the correct order of the original discussion.")
else:
    print("❌ The dataset is NOT in the same order as the original discussion.")


❌ The dataset is NOT in the same order as the original discussion.


This pdf scraping function doesn't seem to be working properly. 

Findings 
Seems like the code from DemocratieMonitor doesn't scrape the full document. While going through and comparing csv files and actualy pdf document, I noticed that some parts (Motie) are missing in the CSV files (idk if that's supposed to be tho??) 
- https://zoek.officielebekendmakingen.nl/h-tk-20232024-89-6.pdf
- (page 13, column 90 in the csv file "speeches 2023-2024"): While the specific speaker was not mentioned in the pdf files, csv filed assigned speaker of the speech to "De heer flash"
- last two sentences of the column 90 are spoken by voorzitter according to the pdf files
- first motie didn't show up in the csv. 
- also some numerical figure is missing as well. (Zij krijgt nr....) 
- what is motie? Is it spoken by Voorzitter or someone else?
- party for 'Minister' is not defined

In [78]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import time
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys

driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
driver.get("https://www.google.com")

In [79]:
button = driver.find_element('id','W0wltc')
button.click()

In [80]:
# Find the search bar using the name of the input field
search_bar = driver.find_element("name", "q")

# Type the search term and hit ENTER
search_bar.send_keys("Lijst van Tweede Kamerleden 2023-heden")

In [81]:
#Click enter!
search_bar.send_keys(Keys.RETURN)

# Wait for some time to let the results load
time.sleep(2) 

#The page will now have made the search!

In [82]:
# Locate the titles and URLs of the search hits.
results = driver.find_elements(by=By.CSS_SELECTOR,value='a h3') #Select all links under the div with id search: these are the search results.

# Extract and print the top 10 hits 
for result in results:
    if len(result.text)>0:
        #This is how to get the parent element in selenium. We want the <a> to get the URL.
        parent_element = result.find_element(by=By.XPATH, value='..')
        
        print(f"{result.text}. {parent_element.get_attribute('href')}")

Lijst van Tweede Kamerleden 2023-heden - Wikipedia. https://nl.wikipedia.org/wiki/Lijst_van_Tweede_Kamerleden_2023-heden
Alle Kamerleden | Tweede Kamer der Staten-Generaal. https://www.tweedekamer.nl/kamerleden_en_commissies/alle_kamerleden
Lijst van Tweede Kamerleden 2021-2023. https://nl.wikipedia.org/wiki/Lijst_van_Tweede_Kamerleden_2021-2023
Fracties | Tweede Kamer der Staten-Generaal. https://www.tweedekamer.nl/kamerleden_en_commissies/fracties
Zetelverdeling Tweede Kamer. https://www.parlement.com/id/vh8lnhronvx6/zetelverdeling_tweede_kamer
Tussentijds vertrek uit de Tweede Kamer. https://www.parlement.com/id/vh8lnhrqszxu/tussentijds_vertrek_uit_de_tweede_kamer
Zo ziet de nieuwe Tweede Kamer eruit. https://www.ewmagazine.nl/nederland/achtergrond/2023/12/zo-ziet-de-nieuwe-tweede-kamer-eruit-1378667/
DENK en Tweede Kamerverkiezingen 2023. https://www.denederlandsegrondwet.nl/id/vm59llsqsfdd/denk_en_tweede_kamerverkiezingen_2023
PVV presenteert kandidatenlijst Tweede .... https://ww

In [83]:
# Initialize WebDriver (e.g., Chrome)
driver = webdriver.Chrome()

# Navigate to the webpage
url = "https://nl.wikipedia.org/wiki/Lijst_van_Tweede_Kamerleden_2023-heden"
driver.get(url)

In [107]:
from selenium import webdriver
from selenium.webdriver.common.by import By

# Initialize WebDriver (e.g., Chrome)
driver = webdriver.Chrome()

# Navigate to the webpage
url = "https://nl.wikipedia.org/wiki/Lijst_van_Tweede_Kamerleden_2023-heden"
driver.get(url)

# Locate the table(s) using XPath or CSS selectors
tables = driver.find_elements(By.XPATH, "//table")  # Locate all tables on the page

# Create a list to store all tables' data
tables_data = []

for table in tables:
    table_data = {}  # Dictionary to store the data for each table
    
    # Extract table headers
    headers = table.find_elements(By.XPATH, ".//thead/tr/th")
    if not headers:  # Fallback if <thead> is not used
        headers = table.find_elements(By.XPATH, ".//tr/th")
    
    header_texts = [header.text for header in headers]
    table_data['headers'] = header_texts  # Store headers in table_data

    # Extract table rows
    rows = table.find_elements(By.XPATH, ".//tbody/tr")
    table_data['rows'] = []  # List to store rows for the table

    for row in rows:
        # Extract columns/cells in each row
        cells = row.find_elements(By.TAG_NAME, "td")
        if cells:
            row_texts = [cell.text for cell in cells]
            table_data['rows'].append(row_texts)  # Append each row's data
    
    tables_data.append(table_data)  # Add the data for this table to the list

# Now tables_data contains all the tables' data in a structured way
# You can access each table's data like this:
for index, table_data in enumerate(tables_data):
    print(f"Table {index + 1}:")
    print("Headers:", table_data['headers'])
    for row in table_data['rows']:
        print(row)

Table 1:
Headers: ['Naam', 'Fractie', 'Begindatum[a]', 'Einddatum', 'Ref']
['Max Aardema', '', 'PVV', '6 december 2023', '', '[2]']
['Thierry Aartsen', '', 'VVD', '6 december 2023', '', '[3]']
['Ismail el Abassi', '', 'DENK', '6 december 2023', '', '[4]']
['Fleur Agema', '', 'PVV', '6 december 2023', '2 juli 2024[b]', '[5]']
['Stephan van Baarle', '', 'DENK', '6 december 2023', '', '[6]']
['Mpanzu Bamenga', '', 'D66', '6 december 2023', '', '[7]']
['Thierry Baudet', '', 'FVD', '6 december 2023', '13 januari 2025[c]', '[8]']
['Bente Becker', '', 'VVD', '6 december 2023', '', '[9]']
['Sandra Beckerman', '', 'SP', '6 december 2023', '', '[10]']
['Harry Bevers', '', 'VVD', '4 juli 2024[d]', '', '[11]']
['Mirjam Bikker', '', 'CU', '6 december 2023', '', '[12]']
['Reinder Blaauw', '', 'PVV', '6 december 2023', '', '[13]']
['Henri Bontenbal', '', 'CDA', '6 december 2023', '', '[14]']
['Diederik Boomsma', '', 'NSC', '4 juli 2024[d]', '', '[15]']
['Maikel Boon', '', 'PVV', '6 december 2023', ''

In [88]:
header = table_data['headers']
header

['Naam', 'Fractie', 'Begindatum[a]', 'Einddatum', 'Ref']

In [105]:
table_data['rows'][:10]

[['Max Aardema', 'PVV', '6 december 2023', '', '[2]'],
 ['Thierry Aartsen', 'VVD', '6 december 2023', '', '[3]'],
 ['Ismail el Abassi', 'DENK', '6 december 2023', '', '[4]'],
 ['Fleur Agema', 'PVV', '6 december 2023', '2 juli 2024[b]', '[5]'],
 ['Stephan van Baarle', 'DENK', '6 december 2023', '', '[6]'],
 ['Mpanzu Bamenga', 'D66', '6 december 2023', '', '[7]'],
 ['Thierry Baudet', 'FVD', '6 december 2023', '13 januari 2025[c]', '[8]'],
 ['Bente Becker', 'VVD', '6 december 2023', '', '[9]'],
 ['Sandra Beckerman', 'SP', '6 december 2023', '', '[10]'],
 ['Harry Bevers', 'VVD', '4 juli 2024[d]', '', '[11]']]

In [98]:
cleaned_data = [] 

for row in table_data['rows']:
    if len(row) < 5: 
        continue 
    if len(row) == 6: 
        del row[1] 
    if len(row) == 5: 
        cleaned_data.append(row) 
for row in cleaned_data:
    print(row) 

['Max Aardema', 'PVV', '6 december 2023', '', '[2]']
['Thierry Aartsen', 'VVD', '6 december 2023', '', '[3]']
['Ismail el Abassi', 'DENK', '6 december 2023', '', '[4]']
['Fleur Agema', 'PVV', '6 december 2023', '2 juli 2024[b]', '[5]']
['Stephan van Baarle', 'DENK', '6 december 2023', '', '[6]']
['Mpanzu Bamenga', 'D66', '6 december 2023', '', '[7]']
['Thierry Baudet', 'FVD', '6 december 2023', '13 januari 2025[c]', '[8]']
['Bente Becker', 'VVD', '6 december 2023', '', '[9]']
['Sandra Beckerman', 'SP', '6 december 2023', '', '[10]']
['Harry Bevers', 'VVD', '4 juli 2024[d]', '', '[11]']
['Mirjam Bikker', 'CU', '6 december 2023', '', '[12]']
['Reinder Blaauw', 'PVV', '6 december 2023', '', '[13]']
['Henri Bontenbal', 'CDA', '6 december 2023', '', '[14]']
['Diederik Boomsma', 'NSC', '4 juli 2024[d]', '', '[15]']
['Maikel Boon', 'PVV', '6 december 2023', '', '[16]']
['Vincent van den Born', 'PVV', '6 december 2023', '', '[17]']
['Martin Bosma', 'PVV', '6 december 2023', '', '[18]']
['Derk 

In [100]:
mp_2023 = pd.DataFrame(cleaned_data, columns = header) 


In [108]:
mp_2023.head(20) 

,Naam,Fractie,Begindatum[a],Einddatum,Ref
0,Max Aardema,PVV,6 december 2023,,[2]
1,Thierry Aartsen,VVD,6 december 2023,,[3]
2,Ismail el Abassi,DENK,6 december 2023,,[4]
3,Fleur Agema,PVV,6 december 2023,2 juli 2024[b],[5]
4,Stephan van Baarle,DENK,6 december 2023,,[6]
5,Mpanzu Bamenga,D66,6 december 2023,,[7]
6,Thierry Baudet,FVD,6 december 2023,13 januari 2025[c],[8]
7,Bente Becker,VVD,6 december 2023,,[9]
8,Sandra Beckerman,SP,6 december 2023,,[10]
9,Harry Bevers,VVD,4 juli 2024[d],,[11]


In [109]:
mp_2023.drop(['Begindatum[a]', 'Einddatum', 'Ref'], axis=1) 

,Naam,Fractie
0,Max Aardema,PVV
1,Thierry Aartsen,VVD
2,Ismail el Abassi,DENK
3,Fleur Agema,PVV
4,Stephan van Baarle,DENK
...,...,...
170,Geert Wilders,PVV
171,Natascha Wingelaar,NSC
172,Dilan Yeşilgöz,VVD
173,Claudia van Zanten,BBB


In [121]:
df.head() 

,file_id,speaker_name,speaker_party,speech_text,jaar,date,kamer,category,title,document_number,url,meta_url,vergadernummer
0,h-tk-20232024-89-6,De heer Bosma,PVV,Aan de orde is de Wijziging Omgevingswet en e...,2023-2024,2024-06-26,tk,handelingen,6 Wet uitvoering verordening ontbossingsvrije ...,NaN,https://zoek.officielebekendmakingen.nl/h-tk-2...,https://zoek.officielebekendmakingen.nl/h-tk-2...,"nr. 89, item 6"
1,h-tk-20232024-89-6,De heer Bosma,PVV,Als eerste geef ik graag het woord aan mevrou...,2023-2024,2024-06-26,tk,handelingen,6 Wet uitvoering verordening ontbossingsvrije ...,NaN,https://zoek.officielebekendmakingen.nl/h-tk-2...,https://zoek.officielebekendmakingen.nl/h-tk-2...,"nr. 89, item 6"
2,h-tk-20232024-89-6,Mevrouw Hertzberger,NSC,Voorzitter. Deze wet voor de implementatie va...,2023-2024,2024-06-26,tk,handelingen,6 Wet uitvoering verordening ontbossingsvrije ...,NaN,https://zoek.officielebekendmakingen.nl/h-tk-2...,https://zoek.officielebekendmakingen.nl/h-tk-2...,"nr. 89, item 6"
3,h-tk-20232024-89-6,De heer Bosma,PVV,Neem even een pauze. Mevrouw Bromet praat die...,2023-2024,2024-06-26,tk,handelingen,6 Wet uitvoering verordening ontbossingsvrije ...,NaN,https://zoek.officielebekendmakingen.nl/h-tk-2...,https://zoek.officielebekendmakingen.nl/h-tk-2...,"nr. 89, item 6"
4,h-tk-20232024-89-6,Mevrouw Bromet,GroenLinks-PvdA,"Ik verstond het niet, maar dat geeft niet, de...",2023-2024,2024-06-26,tk,handelingen,6 Wet uitvoering verordening ontbossingsvrije ...,NaN,https://zoek.officielebekendmakingen.nl/h-tk-2...,https://zoek.officielebekendmakingen.nl/h-tk-2...,"nr. 89, item 6"


In [124]:
df['date'].dtypes 

dtype('O')

In [123]:
df.dtypes

file_id             object
speaker_name        object
speaker_party       object
speech_text         object
jaar                object
date                object
kamer               object
category            object
title               object
document_number    float64
url                 object
meta_url            object
vergadernummer      object
dtype: object

In [125]:
df['date'] = pd.to_datetime(df['date']) 

In [126]:
count_before_date = (df['date'] < "2023-12-06").sum()
print(count_before_date)


15624


<bound method Series.unique of 0       2024-06-26
1       2024-06-26
2       2024-06-26
3       2024-06-26
4       2024-06-26
           ...    
67259   2023-09-26
67260   2023-09-26
67261   2023-09-26
67262   2023-09-26
67263   2023-09-26
Name: date, Length: 67264, dtype: datetime64[ns]>

Findings: 
- Scraping itself is done through wikipedia using selenium and dataset shows name and party
- But the problem is that the csv file also contains documents from before December 6, 2023, when the current tweedekamer was formed
- Which in general requires two scraping per each csv
- Also wikipedia and csv file refers both persons' name and party name differently.
- So I guess the best to use is tweedekamer API? 

In [151]:
import tkapi

api = tkapi.TKApi()
personen = api.get_personen(max_items=50)
for persoon in personen:
    print(persoon.fractielabel) 

AttributeError: 'Persoon' object has no attribute 'fractielabel'

In [134]:
df['Party'].value_counts() 

Party
Unknown    250
Name: count, dtype: int64

In [157]:
import json
import requests

API_URL = "https://gegevensmagazijn.tweedekamer.nl/OData/v4/2.0/Persoon?$expand=FractieZetelPersoon&$filter=Verwijderd eq false"
response = requests.get(API_URL)
data = response.json()

# Print first person's details for inspection
print(json.dumps(data["value"][2], indent=4, ensure_ascii=False))


{
    "Id": "984df8b2-85d5-43cc-b1e8-0045cb0bf65f",
    "Nummer": 1248,
    "Titels": null,
    "Initialen": "HA",
    "Tussenvoegsel": null,
    "Achternaam": "Dambrink",
    "Voornamen": null,
    "Roepnaam": null,
    "Geslacht": "man",
    "Functie": "Oud Kamerlid",
    "Geboortedatum": "1881-03-13",
    "Geboorteplaats": "-",
    "Geboorteland": null,
    "Overlijdensdatum": "1940-10-16",
    "Overlijdensplaats": null,
    "Woonplaats": null,
    "Land": "NL",
    "ContentType": null,
    "ContentLength": null,
    "GewijzigdOp": "2023-08-29T13:09:45+02:00",
    "ApiGewijzigdOp": "2023-08-29T11:16:24.2820522Z",
    "Verwijderd": false,
    "Fractielabel": null,
    "FractieZetelPersoon": []
}
